## The parser class ##

In [272]:


from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON


list_of_prefix = """
PREFIX prop: <http://dbpedia.org/property/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX res: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX db: <http://dbpedia.org/>
PREFIX yago: <http://dbpedia.org/class/yago/>
PREFIX onto: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX dbpedia2: <http://dbpedia.org/property/>
"""


input_query = """
SELECT ?p
WHERE {
   dbr:Albert_Einstein ?p dbr:German_Empire .
}
"""


sparql_o = SPARQL(input_query)


class Triple:  # chaneg name
    def __init__(self, sub, pre, obj):
        self.sub = sub
        self.pre = pre
        self.obj = obj

    def get_subject(self):

        return self.sub

    def get_predicate(self):
        return self.pre

    def get_object(self):
        return self.obj


triples_to_query = []

for s in sparql_o.triple_list:
    triples_to_query.append(
        Triple(s.split(' ')[0], s.split(' ')[1], s.split(' ')[2]))


# for t in triples_to_query:
#     print("sub " + t.get_subject())
#     print("pre " + t.get_predicate())
#     print("obj " + t.get_object())
#     print('------------------')

# # x = a.template

cost = 0

list_of_queries = []




o = "o"
oo ="x"
num = 4
for x in range(num):
    last = triples_to_query[-1]
    triples_to_query.append(Triple(last.get_object(), f"?{oo*(cost+1)}", f"?{o*(cost+1)}"))
    cost += 1

            
variable = "?p "
for t in triples_to_query[1:]:
    variable += f"{t.get_predicate()} {t.get_object()}"
   


sparql_query = f"{sparql_o.intent} {variable} where {{\n"
for triple in triples_to_query:
    sparql_query += f"    {triple.get_subject()} {triple.get_predicate()} {triple.get_object()} .\n"
sparql_query += "}\n LIMIT 20"

# full_query = list_of_prefix + sparql_query


full_querry = list_of_prefix + sparql_query

print(full_querry)
my_query(full_querry)



PREFIX prop: <http://dbpedia.org/property/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX res: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX db: <http://dbpedia.org/>
PREFIX yago: <http://dbpedia.org/class/yago/>
PREFIX onto: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX dbpedia2: <http://dbpedia.org/property/>
SELECT ?p ?x ?o?xx ?oo?xxx ?ooo?xxxx ?oooo where {
    <http://dbpedia.org/resource/Albert_Einstein> ?p <http://dbpedia.org/resource/German_Empire> .
    <http://dbpedia.org/resource/German_Empire> ?x ?o .
    ?o ?xx ?oo .
    ?oo ?xxx ?ooo .
    ?ooo ?xxxx ?oooo .
}
 LIMIT 20


## The endpoint ## 

In [240]:

def my_query(query_to_execute): 
    sparql = SPARQLWrapper("https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
    sparql.setQuery(query_to_execute)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()  
    
    index = results['head']['vars']
    
    result_list = []
    ind_reslut = []
    
    
    for y in index:
        for x in results['results']['bindings']:
            ind_reslut.append(x[y]['value'])
        result_list.append(ind_reslut)
        ind_reslut = []
        
    wr_my_list(result_list)



## The returned result ##

In [ ]:

def execute_query(query_to_execute): 
    sparql = SPARQLWrapper("https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
    sparql.setQuery(query_to_execute)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()  

    index = results['head']['vars']

    result_list = []
    ind_reslut = []


    for y in index:
        for x in results['results']['bindings']:
            ind_reslut.append(x[y]['value'])
        result_list.append(ind_reslut)
        ind_reslut = []

    print(result_list)






## Node forward ## 


In [265]:
node_forward = """
        
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?p ?place ?x  ?y?a ?b
WHERE {
    dbp:Albert_Einstein ?p ?place .
  	?place ?x ?y.
  	?y ?a ?b.
}
LIMIT 10


"""

o = "o"
oo ="oo"
num = 4
for x in range(num):
    last = triples_to_query[-1]
    triples_to_query.append(Triple(last.get_object(), f"?{oo*(cost+1)}", f"?{o*(cost+1)}"))
    cost += 1

            
variable = "?p "
for t in triples_to_query[1:]:
    variable += f"{t.get_predicate()} {t.get_object()}"
   

    



# for t in triples_to_query[1:]:
#     print("sub " + t.get_subject())
#     print("pre " + t.get_predicate())
#     print("obj " + t.get_object())
#     print('------------------')

# what we do is add a new set of SPO to the end of the triples to be executed 



'?p ?oo ?o?oooo ?oo?oooooo ?ooo?oooooooo ?oooo?oooooooooo ?ooooo?oooooooooooo ?oooooo?oooooooooooooo ?ooooooo?oooooooooooooooo ?oooooooo'



## Node Backward ## 


In [12]:
node_backward = """

PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>


SELECT *
WHERE {
  ?a ?b ?z.
  ?z ?p ?x.
  ?x ?y dbp:Albert_Einstein.
    dbp:Albert_Einstein ?place dbr:German_Empire .
}

limit 20
"""

query_to_execute = node_backward

## Substitution ##

In [8]:
sub = """

PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

#SELECT ?p ?place
#WHERE {
#    dbp:Albert_Einstein dbp:birthplace dbr:German_Empire .
#}
#
SELECT ?p ?place
WHERE {
    dbp:Albert_Einstein ?p ?place .
}
LIMIT 10
"""

query_to_execute = sub

## Printer of the list ##

In [9]:
# def print_my_list(l1list):
#     for l1 in zip(l1list):
#         for l2 in l1:
#             print(l2)
#         print()
#         # print(l1,l2)
#         print('------------------')

In [239]:
def wr_my_list(l1list):
    file_path = 'test.txt'
    with open(file_path, 'w') as file:
        
        for l1 in zip(*l1list):
            for l2 in l1:
                file.write(str(l2) + "   " )
            file.write('\n ------------------ \n')


## -------------------------------------------------------------------------------------------------------------------------------------- ##

## question ##
 1. every estimation we create two new variables , what do we do with them so if we start with og we keep going ?

In [ ]:
def answer_convert(item_answer):
    if 'boolean' in item_answer.keys():
        at = 'boolean'
    else:
        at = item_answer['head']['vars'][0]
    answer = []
    if at == 'boolean':
        answer.append(item_answer['boolean'])
    else:
        for cand in item_answer['results']['bindings']:
            if at == 'date':
                answer.append(cand['date']['value'])
            elif at == 'number':
                answer.append(cand['c']['value'])
            elif at == 'resource' or at == 'uri':
                answer.append(cand['uri']['value'])
            elif at == 'string':
                answer.append(cand['string']['value'])
            elif at == 'callret-0':
                answer.append(cand['callret-0']['value'])
            else: 
                answer.append(cand[at]['value'])
    return answer


sparql = SPARQLWrapper(
    "https://api.triplydb.com/datasets/academy/pokemon/services/pokemon/sparql")
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()  
answer = answer_convert(results)

type(results.items())

print(results.keys())
print('boolean' in results.keys())


In [252]:
num = 2

text = "hello"

output = f"this is a {text*num} "

print(output)

this is a hellohello 
